In [ ]:
import wrds
import pandas as pd

In [ ]:
# Step 1: Load your S&P 500 tickers
sp500_df = pd.read_csv('sp500_cik.csv')
tickers = sp500_df['Symbol'].dropna().unique().tolist()

# Step 2: Connect to WRDS
db = wrds.Connection()

# Step 3: Define your date range
start_date = '2000-01-01'
end_date = '2025-01-01'

# Step 4: Convert tickers to upper case (CRSP format)
tickers = [t.upper() for t in tickers]

# Step 5: Create query to pull daily prices, shrout, and market cap
tickers_str = ",".join(f"'{t}'" for t in tickers)

query = f"""
SELECT
    dsf.date,
    msf.ticker,
    msf.comnam,
    dsf.openprc AS open,
    dsf.askhi AS high,
    dsf.bidlo AS low,
    dsf.prc AS close,
    dsf.vol AS volume,
    ABS(dsf.prc) * dsf.shrout * 1000 AS market_cap
FROM crsp.dsf dsf
JOIN crsp.msenames msf ON dsf.permno = msf.permno
    AND dsf.date BETWEEN msf.namedt AND msf.nameendt
WHERE msf.ticker IN ({tickers_str})
    AND dsf.date BETWEEN '{start_date}' AND '{end_date}'
ORDER BY dsf.date, msf.ticker
"""

# Step 6: Run the query and fetch results
sp500_data = db.raw_sql(query)

# Step 7: Optional - sort and clean
sp500_data = sp500_data.sort_values(by=['ticker', 'date'])

# Step 8: Save to CSV
sp500_data.to_csv('sp500_data.csv', index=False)

# Step 9: Close WRDS connection
db.close()